In [1]:
import pandas as pd
from unidecode import unidecode
from re import sub

def format_text(input_string):
    no_a = unidecode(str(input_string)) # Eliminar tildes
    no_sc = sub('[^a-zA-Z0-9]+', '', no_a) # Eliminar caracteres especiales
    return no_sc.lower().strip().replace(' ', '') # Eliminar espacios y pasar todo a minúscula

def format_course_number(input_string):
    if len(input_string) == 1: # Si el número de curso tiene 1 dígito. Ej. COM1
        return f'00{input_string}' # Agrega dos ceros. Ej. COM001

    elif len(input_string) == 2: # Si el número de curso tiene 2 dígitos. Ej. COM01
        return f'0{input_string}' # Agrega un cero. Ej. COM001
    return input_string

In [2]:
cols_courses_global = ['NRC', 'Escuela', 'Materia', 'Número Curso', 'Nombre Curso']
df_courses_global = pd.read_excel('Cursos programados 2024_FINAL.xlsx', sheet_name='Hoja2', dtype=str)[cols_courses_global].astype(str)

df_courses_global[cols_courses_global] = df_courses_global[cols_courses_global].apply(lambda x : x.str.strip())

df_courses_global['Número Curso'] = df_courses_global['Número Curso'].apply(format_course_number)
df_courses_global['Sigla'] = df_courses_global['Materia'] + df_courses_global['Número Curso']
df_courses_global = df_courses_global.drop(['Materia', 'Número Curso'], axis=1)

df_courses_global['Nombre Curso (formateado)'] = df_courses_global['Nombre Curso'].apply(format_text)
df_courses_global['Sigla (formateado)'] = df_courses_global['Sigla'].apply(format_text)

In [5]:
cols_courses_vrai = ['NRC', 'Escuela', 'Nombre Curso', 'Sigla']
df_courses_vrai = pd.read_excel('3. Formulario preselección de cursos intercambio 2024-2. (1-13)_REZAGADOS.xlsx', sheet_name='Catálogo VRAI', dtype=str)[cols_courses_vrai].astype(str)

df_courses_vrai[cols_courses_vrai] = df_courses_vrai[cols_courses_vrai].apply(lambda x : x.str.strip())
df_courses_vrai['Nombre Curso (formateado)'] = df_courses_vrai['Nombre Curso'].apply(format_text)
df_courses_vrai['Sigla (formateado)'] = df_courses_vrai['Sigla'].apply(format_text)

In [7]:
cols_students = ['Nombre completo', 'RUT UC', 'Nombre', 'Sigla', 'NRC']
df_students = pd.read_excel('3. Formulario preselección de cursos intercambio 2024-2. (1-13)_REZAGADOS.xlsx', dtype=str, sheet_name="Cursos generales")[cols_students].astype(str)

df_students[cols_students] = df_students[cols_students].apply(lambda x : x.str.strip())
df_students['Nombre Curso (formateado)'] = df_students['Nombre'].apply(format_text)
df_students['Sigla (formateado)'] = df_students['Sigla'].apply(format_text)

In [9]:
cols_students_dpt = ['Nombre completo', 'RUT UC', 'Nombre', 'Sigla', 'NRC']
df_students_dpt = pd.read_excel('3. Formulario preselección de cursos intercambio 2024-2. (1-13)_REZAGADOS.xlsx', dtype=str, sheet_name='Cursos deportivos')[cols_students].astype(str)

df_students_dpt[cols_students_dpt] = df_students_dpt[cols_students_dpt].apply(lambda x : x.str.strip())
df_students_dpt['Nombre Curso (formateado)'] = df_students_dpt['Nombre'].apply(format_text)
df_students_dpt['Sigla (formateado)'] = df_students_dpt['Sigla'].apply(format_text)

EN CATALOGO O NO

In [16]:
def verify_nrc(df, course_matches, selected_course_name_format, selected_nrc, obs):
    suggested_nrc = ''
    course_match_nrc = df.loc[(df['Nombre Curso (formateado)'] == selected_course_name_format) & (df['NRC'] == selected_nrc)]
    if not len(course_match_nrc):
        if len(course_matches) > 1:
            obs += 'ERROR, se requiere revisión. NRC no válido para nombre de curso indicado y existen múltiples secciones.'
        else:
            suggested_nrc = course_matches['NRC'].values[0]
            obs += 'NRC inválido (sección única). '
        academic_unit = course_matches['Escuela'].values[0]
    else:
        academic_unit = course_match_nrc['Escuela'].values[0]
    
    return obs, suggested_nrc, academic_unit

def verify_sigla(df, selected_sigla_format, course_matches, obs, suggested_nrc, selected_nrc, selected_course_name_format):
    suggested_sigla = ''
    if selected_sigla_format not in course_matches['Sigla (formateado)'].values:
        course_match = df.loc[(df['Nombre Curso (formateado)'] == selected_course_name_format) & (df['NRC'].isin([selected_nrc, suggested_nrc]))]
        if len(course_match):
            suggested_sigla = course_match['Sigla'].values[0]
            obs += 'Sigla inválida. '
    
    return obs, suggested_sigla

def verify_name(df, selected_nrc, selected_sigla_format, obs, vrai_check):
    suggested_course_name, academic_unit, in_catalogue = '', '', ''
    course_match = df.loc[(df['NRC'] == selected_nrc) & (df['Sigla (formateado)'] == selected_sigla_format)]
    if len(course_match) == 1:
        suggested_course_name, academic_unit = course_match['Nombre Curso'].values[0], course_match['Escuela'].values[0]
        obs += 'Nombre inválido. '
        if vrai_check:
            in_catalogue = True
        else:
            in_catalogue = False
    else:
        if vrai_check:
            in_catalogue = False
        else:
            obs += 'ERROR, se requiere revisión. No existe un nombre para NRC y sigla indicados. '
    
    return obs, suggested_course_name, academic_unit, in_catalogue

def verify_nrc_and_sigla(df, course_matches, selected_course_name_format, selected_sigla_format, selected_nrc, suggested_nrc, obs):
    academic_unit = ''
    obs, suggested_nrc, academic_unit = verify_nrc(df, course_matches, selected_course_name_format, selected_nrc, obs)
    obs, suggested_sigla = verify_sigla(df, selected_sigla_format, course_matches, obs, suggested_nrc, selected_nrc, selected_course_name_format)

    return obs, suggested_nrc, suggested_sigla, academic_unit

In [17]:
def verify_course_info(df, dpt):
    columns = ['NOMBRE COMPLETO', 'RUT UC', 'NOMBRE CURSO INDICADO', 'NOMBRE SUGERIDO', 'UNIDAD ACADÉMICA',
                'NRC INDICADO', 'NRC SUGERIDO',  'SIGLA INDICADA', 'SIGLA SUGERIDA', 'EN CATÁLOGO VRAI', 'OBSERVACIONES']
    df_output = pd.DataFrame(columns=columns)

    for index, row in df.iterrows():
        selected_course_name, selected_course_name_format = row['Nombre'], row['Nombre Curso (formateado)']
        selected_sigla, selected_sigla_format, selected_nrc = row['Sigla'], row['Sigla (formateado)'], row['NRC']
        student_name, rut_uc = row['Nombre completo'], row['RUT UC']
        suggested_course_name, suggested_nrc, suggested_sigla, academic_unit, obs, in_vrai_catalogue = '', '', '', '', '', ''

        course_matches = df_courses_vrai.loc[df_courses_vrai['Nombre Curso (formateado)'] == selected_course_name_format]

        # MODIFICAR NRC O SIGLA
        if len(course_matches):
            academic_unit = course_matches['Escuela'].values[0]
            in_vrai_catalogue = True
            obs, suggested_nrc, suggested_sigla, academic_unit = verify_nrc_and_sigla(df_courses_vrai, course_matches, 
                                                                                selected_course_name_format, selected_sigla_format, 
                                                                                selected_nrc, suggested_nrc, obs)
            
        # MODIFICAR NOMBRE
        else:
            obs, suggested_course_name, academic_unit, in_vrai_catalogue = verify_name(df_courses_vrai, selected_nrc, selected_sigla_format, obs, True)
            if not in_vrai_catalogue:
                course_matches = df_courses_global.loc[df_courses_global['Nombre Curso (formateado)'] == selected_course_name_format]
                if len(course_matches):
                    academic_unit = course_matches['Escuela'].values[0]
                    obs, suggested_nrc, suggested_sigla, academic_unit = verify_nrc_and_sigla(df_courses_global, course_matches, selected_course_name_format, 
                                                                                        selected_sigla_format, selected_nrc, suggested_nrc, obs)
                else:
                    obs, suggested_course_name, academic_unit, in_vrai_catalogue = verify_name(df_courses_global, selected_nrc, selected_sigla_format, obs, False)
            
        df_output.loc[len(df_output)] = [student_name, rut_uc, selected_course_name, suggested_course_name, 
                                        academic_unit, selected_nrc, suggested_nrc, selected_sigla, suggested_sigla, in_vrai_catalogue, obs]

    if dpt:
        df_output.to_csv('output_dpt.csv', index=False)
    else:
        df_output.to_csv('output.csv', index=False)

In [18]:
verify_course_info(df_students, dpt=False)
verify_course_info(df_students_dpt, dpt=True)